*Authors:* 

# Lesson 12: Efficient resource usage

*Goals*: understand how a task (e.g. a program) is executed depending on the syntax used and become aware of the resource consumption.


There are generally two kinds of resources that limit any use case:
**computing power (e.g. available CPU)** and **Memory**.

Both influence the usual metric of interest: **time**, or to be more precise: **walltime** (= the time in real life that passes while a software executes a task).

The general relation is easily comparable with a worker digging a hole:
 * A task is completed faster, if done by a stronger worker ("faster" CPU-Core)
 * A task may be completed faster, if done simultaneously by many workers (more CPU-Cores)
 * A larger task may be completed if there is more space to do so per step  (more memory)
 
 Some tasks primarily require a specific resource (so only rely on CPU or memory), while most others can be balanced to make the most of all available resources. 

In [ ]:
# Enfore styleguide (see lesson 11)
%load_ext pycodestyle_magic
%pycodestyle_on

## Measuring resource consumption:
### Time:
The simplest way of measuring runtime is by computing the difference between two timestamps with the package ```time```. Think of this as writing down the time when you start executing a task, doing the task, and then writing down the time at the end.

In [ ]:
import time
import numpy as np

my_data = np.arange(10)

# Start the clock
wall_start_time = time.time()

# This is the task we want to measure
for _ in range(10):
    my_data = np.sqrt(my_data*2)
    time.sleep(.15)  # wait 150 ms

# Record the passed time
wall_time = time.time() - wall_start_time

print("This took", wall_time, "seconds in real life.")

In some cases, especially for parallel applications, it is also important to know how much time the actual computation required (CPU-time).

Walltime not caused by the CPU might be due to reading and writing of data, waiting for input from the user or another process, or simply by the fact that other processes are running on the same CPU and your program has to wait to be scheduled for CPU operations.

Let's have a look at the previous example from this perspective:

In [ ]:
import time
import numpy as np

my_data = np.arange(10)

wall_start_time = time.time()
cpu_start_time = time.process_time()

for _ in range(10):
    my_data = np.sqrt(my_data*2)
    time.sleep(.15)  # wait 150 ms

wall_time = time.time() - wall_start_time
cpu_time = time.process_time() - cpu_start_time

print("This took", wall_time, "seconds in real live.")
print("This consumed", cpu_time, "seconds of cpu-time.")

We see that it still needs approximately the same time to finish, although the actual computation could be done in a much shorter time. The obvious reason here is the `sleep()` routine, which artificially pauses the execution.
Reasons in real life examples are much harder to identify. Yet this is what you have to do, to optimize your programs. 


### Memory


Now let's have a look at memory consumption.
Every object in Python (e.g. an array) requires memory to be stored. We can check an object's memory consumption with the package ``sys``. For NumPy arrays we use the built-in function ```nbytes```.

In [ ]:
import sys
import numpy as np

print(sys.getsizeof(0))  # integer
print(sys.getsizeof(0.0))  # float

In most programming languages, each primitve data type has a fixed size. For Integers that means there is a minimum and maximum that can be represented. In Python 3 this is not true for integers, which can dynamically grow in size:

In [ ]:
print(sys.getsizeof(0))
print(sys.getsizeof(1))
print(sys.getsizeof(1000000000))
print(sys.getsizeof(10000000000))

In [ ]:
myint = 0
print(sys.getsizeof(myint))
myint += 100
print(sys.getsizeof(myint))
myint += 10000000000
print(sys.getsizeof(myint))

Beware: `sys.getsizeof` returns the size of the object, but not necessarily the sum of the objects it references (like the contents of a list):

In [ ]:
mylist = list(range(1000))
print(sys.getsizeof(mylist))
size = 0
for i in mylist:
    size += sys.getsizeof(i)
print(size)

In general, measuring the size of an object is not trivial and a priory not a well-defined task:
- Which members of a object do you count?
- In the example below, the reference to the next, which includes a reference to the next, ... and even has a cycle

In [ ]:
class Node:
    def __init__(self, nxt, content):
        self.next = nxt
        self.content = content

    def print_list(self):
        print(self.content)
        node = self.next
        while node != self:
            print(node.content)
            node = node.next

In [ ]:
single_node = Node(None, "A")
sys.getsizeof(single_node)

In [ ]:
# create a cyclic list
last_node = Node(None, "D")
clist = Node(last_node, "C")
clist = Node(clist, "B")
clist = Node(clist, "A")
last_node.next = clist

clist.print_list()

In [ ]:
sys.getsizeof(clist)

Numpy arrays, in constrast to python lists, have a fixed-size data type that can be controlled precisely. The postfix indicates the size in bit (8 bits = 1 byte):
- 32 bit = 4 bytes (for floats known as single precision)
- 64 bit = 8 bytes (for floats known as double precision)

In [ ]:
np_data1 = np.zeros(100)  # type implicitly inferred
np_data2 = np.zeros(100, dtype=np.int64)
print(np_data1.dtype, np_data1.nbytes)
print(np_data2.dtype, np_data2.nbytes)

In [ ]:
# we can even use less precise types if we want to:
np_data1 = np.zeros(100, dtype=np.float32)
np_data2 = np.zeros(100, dtype=np.int32)
print(np_data1.dtype, np_data1.nbytes)
print(np_data2.dtype, np_data2.nbytes)

In [ ]:
np_data1 = np.zeros(100, dtype=np.float16)
np_data2 = np.zeros(100, dtype=np.int16)
print(np_data1.dtype, np_data1.nbytes)
print(np_data2.dtype, np_data2.nbytes)

A computer's RAM memory limits us in how many and how large objects our program can use during its execution.

## Improve memory usage
### Ranges

In some cases, we do not need actual data but rather a regular sequence of values, e.g. to loop over.
This is where you should use a range() that always has the same small memory footprint:

In [ ]:
import sys

print(sys.getsizeof(range(5)))
print(sys.getsizeof(range(5000)))

for current_value in range(8, 14, 2):
    print("Current value is:", current_value)

Check the documentation and learn what the parameters of range() mean and how you can control the sequence:
https://docs.python.org/3.9/library/functions.html#func-range

### Contiguos memory order

When data is stored into memory, it will be written as a linear sequence of values and is read the same way. 
If you think of a 2D-array, in Fortan logic, that means every column is put there one after another. C logic is exactly the other way around, meaning every row is put there one after another.
When you access this data, it is makes a huge difference in speed if you read the values from memory in the same sequence they were stored in, or if you have to skip around to get the next value.
Numpy uses C logic by default. 

Let's have an example:

In [ ]:
import time
import numpy as np

size = 2000
my_data = np.random.rand(size, size)

# mean value of columns:
time_columns_start = time.time()
for _ in range(100):  # repeat several times
    my_data.mean(axis=0)
time_columns = time.time() - time_columns_start

# mean value of rows:
time_rows_start = time.time()
for _ in range(100):   # repeat several times
    my_data.mean(axis=1)
time_rows = time.time() - time_rows_start

print("By rows:", time_rows)
print("By columns:", time_columns)

The difference here of ~20% does not seem that much, but you can easily imagine more complex examples than computing a mean or examples that require recurring passes over the data where this effect would be even more prominent.

## Caching

For some problems the concept of *caching* (sometimes called *memoization*) is useful: Store the result of a computation for a given input, such that you can reuse it instead of computing it again. One could build a solution with a dictionary, but Python already has tools included:

In [ ]:
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [ ]:
time_start = time.time()
print(fibonacci(28))
print("Time:", time.time() - time_start)

We can use the decorator `functools.cache` to automatically get caching: for each `n` for which we have computed the result already, it does not need to be recomputed:

In [ ]:
import functools


@functools.cache
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [ ]:
time_start = time.time()
print(fibonacci(28))
print("Time:", time.time() - time_start)

If you cannot afford storing the results for all parameters, it might be better to [use a `lru_cache`](https://docs.python.org/3/library/functools.html#functools.lru_cache) that discards the least recently used elements.

## Compiler vs. Interpreter

Every code you write in Python or any other compute language is by itself just text. 
For a computer to execute your code, it needs to be converted to byte-code. The two concepts of doing this is either via a compiler or via an interpreter. Both do the same job, but with a different strategy because they have different purposes.
Research online what exactly this difference is and find an argument, why code run by an interpreter will always be slower than code run after being compiled. 
 * Answer: Interpreter compiles and immediately executes line by line. It only considers what is needed to execute this line and ignores all other context. The purpose is to execute one command given by the user, (e.g. via a command line interpreter like BASH) to do one thing or so-called "scripts" that are a collection of things to do. 
 A Compiler reads the entire code and converts it as total block, hence is able to optimize it for execution.

Python utilizes an interpreter. This is why you can type and immediately run code in this notebook ;-).
This feature is also the most crucial bottleneck in performance of python programs. This is why most concepts for increasing speed are all about bypassing the interpreter.

### Just-In-Time Compiler (JIT):
The package "numba" brings a JIT to the table that we can use to compile a block of code before it is used. This block will be optimized during the compilation, so it should run faster as if read by the interpreter.
This makes sense to do for blocks that do the largest part of a program's work, e.g. a loop that does the same thing over many iterations. 

Let's have a look at a tiny example that computes the value for Pi by sampling random numbers. 

In [ ]:
import time
import numpy as np


def compute_pi(n_rolls):
    n_hits = 0
    for _ in range(n_rolls):
        x_value = np.random.rand()
        y_value = np.random.rand()
        radius = (x_value**2 + y_value**2)**0.5
        if radius <= 1:
            n_hits += 1
    pi_approx = 4*n_hits/n_rolls
    return pi_approx


def time_run(n_rolls):
    wall_start_time = time.time()
    pi_result = compute_pi(n_rolls)
    wall_time = time.time() - wall_start_time
    print("This took", wall_time, "seconds. Pi is:", pi_result)


time_run(100000)
time_run(1000000)

This example does exactly what you would expect: 
 * The more samples you tell it to perform, the more precise it will approximate the correct value for Pi.
 * If you increase the amount of samples by 10x, it needs 10 times as long to finish. 
 
 Now we will use the JIT to compile the function `compute_pi()` which we can do by just writing a so-called "decorator" in front of the function.
 The first time this function is needed by our program, it will be compiled first and afterwards the compiled version is executed.

In [ ]:
import time
import numpy as np
from numba import njit


@njit
def compute_pi(n_rolls):
    n_hits = 0
    for _ in range(n_rolls):
        x_value = np.random.rand()
        y_value = np.random.rand()
        radius = (x_value**2 + y_value**2)**0.5
        if radius <= 1:
            n_hits += 1
    pi_approx = 4*n_hits/n_rolls
    return pi_approx


def time_run(n_rolls):
    wall_start_time = time.time()
    pi_result = compute_pi(n_rolls)
    wall_time = time.time() - wall_start_time
    print("This took", wall_time, "seconds. Pi is:", pi_result)


time_run(100000)
time_run(100000)
time_run(1000000)

Wow, this is way faster than before ;-) (~100x).

But wait: why does it take so much longer when we call the function time_run() the first time? 

 ## How to identify bottlenecks in a code.
 
 If we want to optimize or just speed up our program, we need to know which part is taking how much time. 
 We can then focus on improving the relevant sections that need the most time to complete. 
 For this we can use so-called profiler. "cProfile" is already on-board with any Pyhton distribution, so we can always use it right away. 
 Let's see what information it can give us. 
 
 We will recycle our example from above.

In [ ]:
import cProfile
import numpy as np


def compute_pi(n_rolls):
    n_hits = 0
    for _ in range(n_rolls):
        x_value = np.random.rand()
        y_value = np.random.rand()
        radius = (x_value**2 + y_value**2)**0.5
        if radius <= 1:
            n_hits += 1
    pi_approx = 4*n_hits/n_rolls
    return pi_approx


cProfile.run('compute_pi(1000000)', sort=1)

CProfile tracks which functions were called how many times and how log it took to execute them per call and in sum of all calles.
In the output above, we can see, that rolling random numbers is what takes up approx. half of the time.
With the other half we cannot yet tell in detail because these are not separate functions that cProfile can distinguish.

Let's change that by just splitting up every sub-taks into a function 

In [ ]:
import cProfile
import numpy as np


def compute_radius(x_value, y_value):
    xy_radius = (x_value**2 + y_value**2)**0.5
    return xy_radius


def check_hit(radius):
    if radius <= 1:
        hit = 1
    else:
        hit = 0
    return hit


def compute_pi(n_rolls):
    n_hits = 0
    for _ in range(n_rolls):
        x_value = np.random.rand()
        y_value = np.random.rand()
        radius = compute_radius(x_value, y_value)
        n_hits += check_hit(radius)
    pi_approx = 4*n_hits/n_rolls
    return pi_approx


cProfile.run('compute_pi(1000000)', sort=1)

First, we can see that our program got slower, because the interpreter now has to handle more function calls.
This offset of <1 second will become negligible for programs that run longer than a few seconds ;-).

Now we can distinguish the time components. Rolling random numbers takes ~38% of the total time, computing the radius ~17%, checking if radius is below 1 takes 8%. The remaining 37% fall to everything else, which is only the for-loop (the one-time computation of pi_approx = 4*n_hits/n_rolls is negligible).

Collect ideas where and how you would start to improve this program.

## Problems

### Problem 1
Get the total size of the elements contained in the following list and store it in `total_size`.

In [ ]:
# PROBLEM (2)
import sys  # use a function from the sys module like we did before

list_of_floats = [0.94, 1.56, -0.68, 1.03, 0.47, 0.47, -1.09, -1.96, -0.8, 0.5]

# SOLUTION
total_size = sum(sys.getsizeof(f) for f in list_of_floats)

# PROBLEM-TEST
total_size

In [ ]:
# SELF-CHECK
# should be an integer size and larger that this
type(total_size), total_size > 200

### Problem 2
We want to create a numpy array with three floats (initialized with zeros) like
```python
array = np.zeros(3)
```
Store the size in bytes of `array` (= all three elements) in a variable `size_in_bytes`. Calculate the size in bits from this and store it in `size_in_bits`.

In [ ]:
# PROBLEM (2)
import numpy as np
array = np.zeros(3)

# SOLUTION
size_in_bytes = array.nbytes
size_in_bits = size_in_bytes * 8

# PROBLEM-TEST
size_in_bytes, size_in_bits

In [ ]:
# SELF-CHECK
# these should be filled now
size_in_bytes, size_in_bits

### Problem 3
Create a variable `array` like before (numpy array with 3 floats), but specify the datatype such that *each entry* needs **2 bytes**

In [ ]:
# PROBLEM (2)
import numpy as np

# SOLUTION
array = np.zeros(3, dtype=np.float16)
# PROBLEM-TEST
array.size, array.nbytes

In [ ]:
# SELF-CHECK
# it should contain 3 elements
print("Number of elements:", array.size)
# this should be the size in bytes of *all* three entries
print("Size in bytes of all elements:", array.nbytes)

### Problem 4
Which of these statements is true?

A) CPU time $\ge$ wall time  
B) CPU time $\le$ wall time  
C) CPU time and wall time are synonyms  
D) CPU time is something only system admins need to care about

In [ ]:
# PROBLEM (2)
# store your answer as a string (A/B/C/D) in a variable named x
# SOLUTION
x = "B"
# PROBLEM-TEST
x.strip().lower()

In [ ]:
# SELF-CHECK
x in ["A", "B", "C", "D"]

### Problem 5
Which of these statements is true?

A) Interpreted code is usually faster than compiled code.  
B) Compiling code is always the best solution for everything that you do.  
C) Compiled code is usually faster than interpreted code.  
D) Compiling code is never good, because the compilation time adds additional overhead.  

In [ ]:
# PROBLEM (2)
# store your answer as a string (A/B/C/D) in a variable named x
# SOLUTION
x = "C"
# PROBLEM-TEST
x.strip().lower()

In [ ]:
# SELF-CHECK
x in ["A", "B", "C", "D"]